In [13]:
import pandas as pd

# Step 1: Load the CSV file into a DataFrame
file_path = '/Users/annelise/Documents/GitHub/Wine_tasting_KG/data_kaggle/'
wine_data = pd.read_csv(file_path + 'winemag-data-130k-v2.csv')

# Display the shape of the DataFrame
print("Original data shape:", wine_data.shape)

# Step 2: Data Cleaning and Preprocessing

# Define the columns to keep
columns_to_keep = ['country', 'description', 'points', 'price', 'title', 'province', 'variety', 'winery', 'taster_name']
wine_data_sel = wine_data[columns_to_keep]
print("Selected data shape:", wine_data_sel.shape)

# Check for missing values
missing_values = wine_data_sel.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Drop rows where any of the specified columns have missing values
wine_data_cleaned = wine_data_sel.dropna()
print("Shape of cleaned data (after dropping NAs):", wine_data_cleaned.shape)

# Step 3: Handling Duplicates

# Define criteria for identifying duplicates
duplicate_criteria = ['country', 'province', 'variety', 'winery', 'taster_name', 'title']

# Identify perfect duplicates (where all columns match, including description, price, and points)
perfect_duplicates = wine_data_cleaned[wine_data_cleaned.duplicated(keep='first')]
print(f"Number of perfect duplicates: {perfect_duplicates.shape[0]}")

# Remove perfect duplicates, keeping only the first occurrence
wine_data_cleaned = wine_data_cleaned.drop_duplicates(keep='first')

# Identify partial duplicates based on key characteristics (excluding 'description', 'price', and 'points')
partial_duplicates = wine_data_cleaned[wine_data_cleaned.duplicated(subset=duplicate_criteria, keep=False)]

# Initialize a list to store merged records
merged_records = []

# Process partial duplicates
if not partial_duplicates.empty:
    print("Found partial duplicates (same key characteristics but different description, price, or points):")
    partial_duplicates_sorted = partial_duplicates.sort_values(by=duplicate_criteria)
    partial_duplicates_grouped = partial_duplicates_sorted.groupby(duplicate_criteria)

    for name, group in partial_duplicates_grouped:
        # Merge descriptions, average points and price
        merged_description = " ".join(group['description'].unique())
        average_points = group['points'].mean()
        average_price = group['price'].mean()
        
        # Create a merged record
        merged_row = {
            'country': group['country'].iloc[0],
            'province': group['province'].iloc[0],
            'variety': group['variety'].iloc[0],
            'winery': group['winery'].iloc[0],
            'taster_name': group['taster_name'].iloc[0],
            'title': group['title'].iloc[0],
            'description': merged_description,
            'points': average_points,
            'price': average_price
        }
        merged_records.append(merged_row)
    
    # Remove the original partial duplicates from the cleaned data
    wine_data_cleaned = wine_data_cleaned.drop_duplicates(subset=duplicate_criteria, keep=False)

    # Add the merged rows back into the cleaned data
    merged_df = pd.DataFrame(merged_records)
    wine_data_cleaned = pd.concat([wine_data_cleaned, merged_df], ignore_index=True)

    print("Shape of cleaned data (after combining partial duplicates):", wine_data_cleaned.shape)

else:
    print("No partial duplicates found based on the specified criteria.")

# Final shape of the cleaned dataset
print("Final shape of cleaned data:", wine_data_cleaned.shape)

# Save the cleaned and processed data to a new CSV file
wine_data_cleaned.to_csv(file_path + 'cleaned_wine_data.csv', index=False)


Original data shape: (129971, 14)
Selected data shape: (129971, 9)
Missing values in each column:
 country           63
description        0
points             0
price           8996
title              0
province          63
variety            1
winery             0
taster_name    26244
dtype: int64
Shape of cleaned data (after dropping NAs): (96420, 9)
Number of perfect duplicates: 8155
Found partial duplicates (same key characteristics but different description, price, or points):
Shape of cleaned data (after combining partial duplicates): (87706, 9)
Final shape of cleaned data: (87706, 9)
